Q1. Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use from keras.datasets import fashion_mnist for getting the fashion mnist dataset.

In [4]:
import pandas as pd
import numpy as np
from keras.datasets import fashion_mnist
import wandb

In [5]:
wandb.init(project="Assignment - 1", entity="da24m014-iit-madras")

# Load the Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()



# Class names for Fashion-MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Log sample images to wandb
sample_images = []
unique_classes = np.unique(y_train)

for cls in unique_classes:
    sample_idx = np.where(y_train == cls)[0][0]  # Find an example for the class
    img = x_train[sample_idx]
    
    sample_images.append(wandb.Image(img, caption=class_names[cls]))

# Log images to wandb
wandb.log({"Sample Images": sample_images})

# Finish wandb run
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: da24m014 (da24m014-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1)
num_classes = 10

In [7]:
def one_hot_encode(y, num_classes):
    encoded = np.zeros((y.size,num_classes))
    encoded[np.arange(y.size),y] = 1
    return encoded

y_train, y_test = one_hot_encode(y_train,num_classes), one_hot_encode(y_test, num_classes)

In [8]:
split_idx = int(0.9*len(x_train))
x_train, x_val = x_train[:split_idx], x_train[split_idx:]
y_train, y_val = y_train[:split_idx], y_train[split_idx:]

In [9]:
# Activation Functions
def relu(Z):
    return np.maximum(0, Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def tanh(Z):
    return np.tanh(Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)

activation_functions = {"relu": relu, "sigmoid": sigmoid, "tanh": tanh}

In [10]:
# Optimizers
def optimizer_update(weights, biases, grads_W, grads_b, learning_rate, optimizer, velocity, momentum=0.9):
    if optimizer == "sgd":
        for i in range(len(weights)):
            weights[i] -= learning_rate * grads_W[i]
            biases[i] -= learning_rate * grads_b[i]
    elif optimizer == "momentum":
        for i in range(len(weights)):
            velocity[i] = momentum * velocity[i] - learning_rate * grads_W[i]
            weights[i] += velocity[i]
            biases[i] -= learning_rate * grads_b[i]
    elif optimizer == "nesterov":
        for i in range(len(weights)):
            prev_velocity = velocity[i]
            velocity[i] = momentum * velocity[i] - learning_rate * grads_W[i]
            weights[i] += -momentum * prev_velocity + (1 + momentum) * velocity[i]
            biases[i] -= learning_rate * grads_b[i]
    elif optimizer == "rmsprop":
        epsilon = 1e-8
        decay_rate = 0.99
        for i in range(len(weights)):
            velocity[i] = decay_rate * velocity[i] + (1 - decay_rate) * (grads_W[i] ** 2)
            weights[i] -= learning_rate * grads_W[i] / (np.sqrt(velocity[i]) + epsilon)
            biases[i] -= learning_rate * grads_b[i]
    elif optimizer == "adam" or optimizer == "nadam":
        beta1, beta2, epsilon = 0.9, 0.999, 1e-8
        for i in range(len(weights)):
            velocity[i] = beta1 * velocity[i] + (1 - beta1) * grads_W[i]
            velocity_corrected = velocity[i] / (1 - beta1)
            biases[i] -= learning_rate * grads_b[i]
            weights[i] -= learning_rate * velocity_corrected

In [15]:
class NeuralNetwork:
    def __init__(self,layers, learning_rate=0.001,activation="relu",optimizer="sgd",weight_init="random"):
        self.layers = layers
        self.activation = activation
        self.optimizer = optimizer
        self.learning_rate = learning_rate
        self.weight_init = weight_init
        self.init_weights(weight_init)
        self.velocity = [np.zeros_like(W) for W in self.weights]

    def init_weights(self, method):
        self.weights = []
        self.biases = []
        for i in range(len(self.layers) - 1):
            if method == "xavier":
                limit = np.sqrt(6 / (self.layers[i] + self.layers[i+1]))
            else:  # Default to "random"
                limit = 0.1
            W = np.random.uniform(-limit, limit, (self.layers[i], self.layers[i+1]))
            self.weights.append(W)
            self.biases.append(np.zeros((1, self.layers[i+1])))

    def forward(self, X):
        self.A = [X]
        for i in range(len(self.weights) - 1):
            Z = self.A[-1] @ self.weights[i] + self.biases[i]
            A = activation_functions[self.activation](Z)
            self.A.append(A)
        Z = self.A[-1] @ self.weights[-1] + self.biases[-1]
        A = softmax(Z)
        self.A.append(A)
        return A
    
    def compute_loss(self, y_true, y_pred):
        return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))
    
    def backward(self, X, y):
        grads_W, grads_b = [], []
        if not hasattr(self, 'A') or len(self.A) == 0:
            raise ValueError("Activations A are not stored correctly. Check forward propagation.")

        dA = self.A[-1] - y
        for i in reversed(range(len(self.weights))):
            dW = self.A[i].T @ dA / X.shape[0]
            db = np.sum(dA, axis=0, keepdims=True) / X.shape[0]
            grads_W.append(dW)
            grads_b.append(db)
            if i > 0:
                dA = (dA @ self.weights[i].T) * (self.A[i] > 0)
        return grads_W[::-1], grads_b[::-1]
    

    def train(self, X_train, y_train, X_test, y_test, epochs, batch_size):
        for epoch in range(epochs):
            y_pred= self.forward(x_train)
            train_loss = self.compute_loss(y_train, y_pred)
            train_accuracy = np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_train,axis=1))
            grads_W, grads_b = self.backward(X_train, y_train)
            optimizer_update(self.weights, self.biases, grads_W, grads_b, self.learning_rate, self.optimizer, self.velocity)
            y_pred_val = self.forward(x_val)
            val_loss = self.compute_loss(y_val, y_pred_val)
            val_accuracy = np.mean(np.argmax(y_pred_val, axis=1) == np.argmax(y_val, axis=1))
            wandb.log({"epoch": epoch+1,"loss": train_loss,"accuracy":train_accuracy,"val_loss": val_loss,"val_accuracy": val_accuracy})





In [16]:
sweep_config = {
    'method': 'random',
    'metric': {'name': 'loss', 'goal': 'minimize'},
    'parameters': {
        'learning_rate': {'values': [0.001, 0.01, 0.1]},
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'values': [5, 10]},
        'hidden_layers': {'values': [2, 3]},
        'hidden_size': {'values': [32, 64]},
        'activation': {'values': ['relu', 'sigmoid']},
        'optimizer': {'values': ['sgd', 'momentum']},
        'weight_init': {'values': ['random', 'xavier']}
    }
}

# wandb.init(project="Assignment - 1")  # Ensure WandB is initialized before using config
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: z5v1zv1m
Sweep URL: https://wandb.ai/da24m014-iit-madras/uncategorized/sweeps/z5v1zv1m


In [17]:
def train_with_wandb():
    wandb.init(project="Assignment - 1")  # Ensure WandB is initialized before using config
    config = wandb.config
    model = NeuralNetwork([784] + [config.hidden_size] * config.hidden_layers + [10],
                          learning_rate=config.learning_rate,
                          activation=config.activation,
                          optimizer=config.optimizer,
                          weight_init=config.weight_init)
    model.train(x_train, y_train, x_test, y_test, config.epochs, config.batch_size)

wandb.agent(sweep_id, function=train_with_wandb)

wandb: Agent Starting Run: 205tiuxp with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_layers: 3
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd
wandb: 	weight_init: random


accuracy,▁▁▁▁▁
epoch,▁▃▅▆█
loss,█▆▄▃▁
val_accuracy,▁▁▁▁▁
val_loss,█▆▄▃▁
accuracy,0.1003
epoch,5
loss,2.31052
val_accuracy,0.09733
val_loss,2.31307


wandb: Agent Starting Run: p5rwy30i with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_layers: 3
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.001
wandb: 	optimizer: momentum
wandb: 	weight_init: random


accuracy,▁▁▂▂▂▃▅▆▇█
epoch,▁▂▃▃▄▅▆▆▇█
loss,██▇▇▆▅▄▃▂▁
val_accuracy,▁▁▂▂▃▅▆▇▇█
val_loss,██▇▇▆▅▄▃▂▁
accuracy,0.12348
epoch,10
loss,2.30111
val_accuracy,0.1245
val_loss,2.30101


wandb: Agent Starting Run: 3t8xhcft with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layers: 3
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.01
wandb: 	optimizer: momentum
wandb: 	weight_init: random


...

accuracy,▁▂▄▆█
epoch,▁▃▅▆█
loss,█▆▄▃▁
val_accuracy,▁▂▄▆█
val_loss,█▆▄▃▁
accuracy,0.11041
epoch,5
loss,2.29875
val_accuracy,0.117
val_loss,2.29787


wandb: Agent Starting Run: 9yr6m7fp with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layers: 3
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.1
wandb: 	optimizer: momentum
wandb: 	weight_init: xavier


accuracy,▁▁▂▄▅▅▆▆▆█
epoch,▁▂▃▃▄▅▆▆▇█
loss,██▇▇▆▅▄▃▂▁
val_accuracy,▁▂▃▄▅▅▅▅▇█
val_loss,██▇▆▆▅▃▃▁▁
accuracy,0.55793
epoch,10
loss,1.19563
val_accuracy,0.61333
val_loss,1.08867


wandb: Agent Starting Run: aznflo78 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layers: 2
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd
wandb: 	weight_init: random


: 